In [2]:
from functools import reduce
from json import loads

import pandas as pd
import requests
from fake_useragent import UserAgent

In [85]:
ua = UserAgent()
url = "https://web.archive.org/web/20240621070124/https://cdn.akamai.steamstatic.com/steam/publicstats/contentserver_bandwidth_stacked.jsonp?v=06-13-2024-07"
headers = {"User-Agent": ua.random}

response = requests.get(url, headers=headers)
startidx = response.text.find("(")
endidx = response.text.find(")")
data = loads(response.text[startidx + 1 : endidx])
series_list = loads(data["json"])

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [83]:
df_list = []
for series in series_list:
    df_dict = {}
    region = series["label"]
    df_dict["Timestamp"] = [
        pd.to_datetime(x[0], unit="ms") for x in series["data"]
    ]
    df_dict[region] = [int(x[1]) for x in series["data"]]
    df_list.append(pd.DataFrame(df_dict))

df = reduce(
    lambda x, y: pd.merge(x, y, on="Timestamp", how="outer"), df_list
)
df = df.sort_values("Timestamp").reset_index(drop=True)

In [84]:
df

,Timestamp,Central America,Africa,Middle East,Oceania,South America,Russia,Asia,Europe,North America
0,2025-01-03 06:30:00,82,56,280,574,752,828,7745,1194,5262
1,2025-01-03 06:40:00,78,56,280,574,698,828,7745,1194,5058
2,2025-01-03 06:50:00,78,56,287,578,698,855,7778,1232,5058
3,2025-01-03 07:00:00,73,57,293,579,652,890,7880,1252,4860
4,2025-01-03 07:10:00,67,58,299,579,617,928,7930,1291,4670
...,...,...,...,...,...,...,...,...,...,...
283,2025-01-05 05:40:00,92,42,241,551,1277,580,9044,1166,7060
284,2025-01-05 05:50:00,90,42,241,555,1203,602,9230,1121,6888
285,2025-01-05 06:00:00,88,44,250,549,1134,616,9338,1104,6698
286,2025-01-05 06:10:00,87,46,258,549,1069,641,9515,1099,6595


In [75]:
df.loc[df["Timestamp"] == pd.Timestamp("2024-11-05 23:00:00")]

,Timestamp,Central America,Africa,Middle East,Oceania,South America,Russia,Asia,Europe,North America
